# FoCS Project - Solution

You have to work on the [Kiva](https://drive.google.com/file/d/1-tJtnIbo1Rt-F1XfoWGVkmBXiI-ciuRx/view) dataset. Some information on the datasets are on the [Kaggle](https://www.kaggle.com/) web page.

## Task 1

*Normalize the `loan_lenders` table. In the normalized table, each row must have one `loan_id` and one lender.*


The first step is to read the `loan_lenders.csv` file and create a dataframe with unnormalized data. Since the file is very large, I introduced a bollean variable `FINAL` - if it is `false` then only a part of data is processed in order to spead up the work. Libraries `panda` and `numpy` will be used for data processing while the efficiency will be measured using the `time` library.

In [3]:
import pandas as pd
import numpy as np
import time as tm
FINAL = False
start_time = tm.time()
loans_lenders_un = pd.read_csv('./data/loans_lenders.csv')
if not FINAL:
    loans_lenders_un = loans_lenders_un.head(10000) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_lenders_un

--- 37.157421827316284 seconds ---


,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
9995,45940,"helga4707, james6963, jimjams, andreas2382, si..."
9996,247491,"priyaram, christian9832, john9242, sandra1434,..."
9997,345274,"priyaram, nicola1093, bobby9744, simon7848, di..."
9998,125945,"joseph1859, matt5349, reese3555, stanley3312, ..."


In [4]:
start_time = tm.time()
pairs_loan_lander = []
for index, row in loans_lenders_un.iterrows(): 
    lenders = row['lenders'].split(',')
    for l in lenders:
        pairs_loan_lander.append({ 'loan_id' : row['loan_id'], 'lender': l.strip() })
loans_lenders = pd.DataFrame(pairs_loan_lander) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_lenders 

--- 3.2877705097198486 seconds ---


,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
245000,225434,wongacom3393
245001,225434,marleneanddel8151
245002,225434,joanne4956
245003,225434,juddie7070


## Task 2

*For each loan, add a column `duration` corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.*

Firstly, I'll read the `loans.csv` file.


In [5]:
start_time = tm.time()
loans = pd.read_csv('./data/loans.csv')
if not FINAL:
    loans = loans.head(20000)
time = tm.time() -start_time
print("--- %s seconds ---" % time) 
loans

MemoryError: 

In order to see the exact titles of the columns and their values I'll do as followes:

In [ ]:
loans.columns

In [ ]:
loans['planned_expiration_time']


In [ ]:
loans['disburse_time']

A `duration` coulmn is created and filled with `None`. 
I'm going now through every row of the dataframe `loans`, and pick the values of the `planned_expiration_time` and `disburse_time`. 
In order to check whether there are missing values I'm using the `pd.notna` method. 
Since in both columns the time is recorded as `str`, a `strptime` method from the `datetime` library is used to transform strings into date-time format. Subsequently, the value of the `disburse_time` is subtracted from the value of the `planned_expiration_time`, and `days` metod gives the value of this difference converted in days. 
Finally, a restricted view of the resulting dataframe is displayed. 

In [ ]:
from datetime import datetime as dt
start_time = tm.time()
loans['duration'] = None
for index, row in loans.iterrows(): 
    s2 = row['planned_expiration_time']
    s1 = row['disburse_time']
    if( pd.notna(s1) and pd.notna(s2) and s1 != '' and s2 != ''):
        d2 = dt.strptime(s2, "%Y-%m-%d %H:%M:%S.%f %z")
        d1 = dt.strptime(s1, "%Y-%m-%d %H:%M:%S.%f %z")
        loans.loc[index,'duration'] = (d2 - d1).days 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_restricted = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time', 'duration'] ]
loans_restricted

Hovewer, much efficient way of solving this task is to create an auxilliary function `daysbetween` which transforms input parameters from strings into date-time, and calulate the difference in days. Then the function is applied to each row of the `loans` dataframe. 

In [ ]:
from datetime import datetime as dt

def daysbetween(s1,s2):
    if( pd.notna(s1) and pd.notna(s2) and s1 != '' and s2 != ''):
        d2 = dt.strptime(s2, "%Y-%m-%d %H:%M:%S.%f %z")
        d1 = dt.strptime(s1, "%Y-%m-%d %H:%M:%S.%f %z")
        return (d2 - d1).days 
    else:
        return None

start_time = tm.time()
loans['duration'] = loans.apply(lambda x: daysbetween(x['disburse_time'],x['planned_expiration_time']), axis=1)
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_restricted = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time', 'duration'] ]
loans_restricted

## Task 3

*Find the lenders that have funded at least twice.*

For this analysis, the dataframe `loans-lenders`, resulting from the first task. Lenders that have corresponding two or more `loan_id` have funded at least twice. 

In [ ]:
start_time = tm.time()
groups = loans_lenders.groupby("lender").lender.count()
groups
lenders_2more = []
for x in groups.iteritems(): 
    if x[1]  >= 2:
        lenders_2more.append(x)
print("--- %s seconds ---" % (tm.time()-start_time)) 
lenders_2more

## Task 4

*For each country, compute how many loans have involved that country as borrowers.*

From the dataframe `loans` I'm grouping by `country_name` and counting the `loan_id`.

In [ ]:
start_time = tm.time()
countries = loans.groupby("country_name").country_name.count()
print("--- %s seconds ---" % (tm.time()-start_time)) 
countries

## Task 5

*For each country, compute the overall amount of money borrowed.*

Again, the dataframe `loans` is processed. After grouping by `country_name`, the `loan_amount` is summed.

In [ ]:
start_time = tm.time()
amount_per_country = loans.groupby("country_name").loan_amount.sum()
print("--- %s seconds ---" % (tm.time()-start_time)) 
amount_per_country 

## Task 6

*Like the previous point, but expressed as a percentage of the overall amount lent* 

First the total amount is calculated. A helper function `calculate_percent` is created in order to calculate the percentage `loan_amount/total_amount*100`, and it is then applied to each row of the series `amount_per_country`.

In [ ]:
start_time = tm.time()
total_amount = loans.loan_amount.sum()

def calculate_percent(v):
    return (v / total_amount) * 100

percentage_per_country= amount_per_country.apply(lambda x: calculate_percent(x))
print("--- %s seconds ---" % (tm.time()-start_time)) 
percentage_per_country

## Task 7

*Like the three previous points, but split for each year (with respect to disburse time).*

Firstly, an auxiliary function `year2` is made in order to enable extracting year from the date.  Subsequently this function is applied on each row of the `loans` dataframe enabling creation of new coulmn `disburse_year`.

In [ ]:
start_time = tm.time()
from datetime import datetime as dt
def year2 (s):
    if( pd.notna(s) and s != ''):
        d = dt.strptime(s, "%Y-%m-%d %H:%M:%S.%f %z")
        return d.year
    return None
    
loans["disburse_year"] = loans.disburse_time.map(lambda x: year2(x))
loans



### Task 7.1

*For each country, compute how many loans have involved that country as borrowers per year.*

In [ ]:
start_time = tm.time()
countries_per_year = loans.groupby(["country_name", "disburse_year"]).country_name.count()
countries_per_year
print("--- %s seconds ---" % (tm.time()-start_time)) 
countries_per_year

### Task 7.2

*For each country, compute the overall amount of money borrowed per year.*

In [ ]:
amount_per_country_per_year = loans.groupby(["country_name","disburse_year"]).loan_amount.sum()
amount_per_country_per_year                                    


### Task 7.3

*Like the previous point, but expressed as a percentage of the overall amount lent per year.* 

Firstly, `total_amount_per_year` is calculated.

In [ ]:
total_amount_per_year = loans.groupby("disburse_year").loan_amount.sum()
total_amount_per_year

An auxiliary function `find_total_per_year` is created in order to enable accessing the total amounts for each year, needed later to calculate the percentage. 
We iterate through `amount_per_country_per_year`, extract the loan amounts grouped by country and year, anc calculate the percentage. As a result, there is a list of dictionaries then converted in dataframe. 

In [ ]:
def find_total_per_year(y):
    for x in total_amount_per_year.items():
        if x[0] == y:
            return x[1]
    return 0

result = []
start_time = tm.time()
for x in amount_per_country_per_year.items():
    v = x[1]
    y = x[0][1]
    c = x[0][0]
    tot = find_total_per_year(y)
    perc = (v / tot) * 100
    result.append({"country_name":c, "disburse_year":y, "percent":perc })
result = pd.DataFrame(result)
print("--- %s seconds ---" % (tm.time()-start_time)) 
result

## Task 8

*For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount*

Two dataframes, `loans` and `loans_lenders` are merged.
The resulting dataframe `number_of_lenders_by_loan_id` has the number of lenders per each loan.
The amount per lender is calculated assuming ttah each lender contributed the same to the loan amount. 
Finally, in order to obtain the total sum per lender, the dataframe is gro

In [ ]:
start_time = tm.time()
merged = pd.merge(left=loans_lenders,right=loans, left_on="loan_id", right_on="loan_id")
merged = merged[["loan_id","lender", "loan_amount"]]
number_of_lenders_by_loan_id = merged.groupby(["loan_id"], as_index = False)[["lender"]].count()
number_of_lenders_by_loan_id
merged2 = pd.merge(left=merged,right=number_of_lenders_by_loan_id, left_on="loan_id", right_on="loan_id")
merged2["amount_per_lender"] = merged2.loan_amount / merged2.lender_y
merged2
lent_by_lender = merged2.groupby(["lender_x"], as_index = False)[["amount_per_lender"]].sum()
print("--- %s seconds ---" % (tm.time()-start_time)) 
lent_by_lender

## Task 9

*For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.*

The file `lenders.csv` contains data related to lenders. `lender_states` is a newely created dataframe containing `country_code` and `permanent_name`.

In [ ]:
start_time = tm.time()
lenders = pd.read_csv("./data/lenders.csv")
if not FINAL:
    lenders = lenders.head(20000) 
lender_states = lenders[["permanent_name","country_code"]]
print("--- %s seconds ---" % (tm.time()-start_time)) 
lender_states

The dataframe `lenders_in_states` contains data related to lenders whose country is known. 

In [ ]:
lenders_in_states = lender_states.loc[lender_states['country_code'].notnull()]
lenders_in_states

The dataframe `lenders_without_states` contains data related to lenders whose country is unknown or the data is missing. 

In [ ]:
lenders_without_states = lender_states.loc[lender_states["country_code"].isnull()]
lenders_without_states

In order to calculate the participation of each country in total amount of lent loans, the following calculation is done:

In [ ]:
start_time = tm.time()
lent_and_state = pd.merge(left=lent_by_lender, right=lenders_in_states, left_on='lender_x', right_on='permanent_name')
lent_total = lent_and_state['amount_per_lender'].sum()
lent_by_state = lent_and_state.groupby(['country_code'], as_index = False\
                                      ).agg({'amount_per_lender':['count', 'sum', \
                                                               lambda x: x.sum()/lent_total]})
lent_by_state.columns = ['country_code', 'lent_count', 'lent_sum', 'lent_factor']
print("--- %s seconds ---" % (tm.time()-start_time)) 
lent_by_state

Total amount of lent money for the lenders with unknown/unavailable country is being calculated

In [ ]:
lent_no_state = pd.merge(left=lent_by_lender, right=lenders_without_states, \
                         left_on='lender_x', right_on='permanent_name')
lent_no_state_total = lent_no_state['amount_per_lender'].sum()
lent_no_state_total

This amount is factorized as previously calculated, and added to `lent_by_state` datafrime as requested in the task.

In [ ]:
lent_by_state['additional_lent'] = lent_by_state.lent_factor * lent_no_state_total
lent_by_state

`loan_and_state` is the dataframe obtained by reducing the `loans` dataframe to the following three columns:

In [ ]:
loan_and_state = loans[['loan_id','country_code','loan_amount']]
loan_and_state

The loan amount per contry is calculated.

In [ ]:
start_time = tm.time()
loan_by_state = loan_and_state.groupby(['country_code'], as_index = False).loan_amount.sum()
loan_by_state.columns = ['country_code','loan_amount_sum']
print("--- %s seconds ---" % (tm.time()-start_time)) 
loan_by_state

Two data frames relating to loaned and lent amounts per country are joined (left join since there are much more loans), the difference is calculated and finally, the entry with maximum difference is displayed.

In [ ]:
start_time = tm.time()
loan_and_lent = pd.merge(loan_by_state, lent_by_state, how='left', on=['country_code'])
loan_and_lent['difference'] = loan_and_lent.loan_amount_sum - loan_and_lent.lent_sum - loan_and_lent.additional_lent 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loan_and_lent[loan_and_lent.difference == loan_and_lent.difference.max()]


## Task 10

*Which country has the highest ratio between the difference computed at the previous point and the population?*

The data related to population are contained in the file `country_stat.csv`

In [6]:
start_time = tm.time()
country_stat = pd.read_csv('./data/country_stats.csv')
country_stat = country_stat[['country_code', 'country_name', 'population', 'population_below_poverty_line']]
country_stat = country_stat.dropna(subset=['population_below_poverty_line'])
print("--- %s seconds ---" % (tm.time()-start_time)) 
country_stat

--- 1.859508991241455 seconds ---


,country_code,country_name,population,population_below_poverty_line
0,IN,India,1339180127,21.9
1,NG,Nigeria,190886311,70.0
2,MX,Mexico,129163276,46.2
3,PK,Pakistan,197015955,29.5
4,BD,Bangladesh,164669751,31.5
...,...,...,...,...
147,MT,Malta,430835,16.3
148,MV,Maldives,436330,16.0
149,ME,Montenegro,628960,8.6
150,TM,Turkmenistan,5758075,0.2


In [7]:
country_stat_loan_and_lent = pd.merge(left=country_stat, right=loan_and_lent, \
                                      left_on='country_code', right_on='country_code')
country_stat_loan_and_lent = country_stat_loan_and_lent[['country_code', 'country_name', 'difference', \
                                                         'population', 'population_below_poverty_line']]
country_stat_loan_and_lent['ratio_1'] = country_stat_loan_and_lent.difference/country_stat_loan_and_lent.population
print("--- %s seconds ---" % (tm.time()-start_time)) 
country_stat_loan_and_lent[country_stat_loan_and_lent.ratio_1 == country_stat_loan_and_lent.ratio_1.max()]

NameError: name 'loan_and_lent' is not defined